In [3]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=r"C:\Users\atulk\OneDrive\Desktop\MAT_PROJ\.env", override=True)

True

In [8]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)
     # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

In [6]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG Application Golden Dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

View the evaluation results for experiment: 'gpt-4o-0110c771' at:
https://smith.langchain.com/o/0262bc87-4e7f-4047-82e4-1316f23416b0/datasets/5aa34710-6f81-401b-a995-014e4277f4ce/compare?selectedSessions=75a046a6-b40a-4e6b-a912-c3347348e5ab




c:\Users\atulk\OneDrive\Desktop\MAT_PROJ\proj1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
20it [00:54,  2.72s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,4.263857,1fe05f92-dff4-4b3d-a0fd-c40cd17a764d,d12d2459-18bc-47eb-8ed1-4f4325cb3205
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,1,3.577859,443a5bdb-ddf7-47af-b34f-f8aca324375b,c83c213d-acb4-4dbd-b5bc-7a4cb95d8093
2,Can LangSmith be used for finetuning and model...,LangSmith is designed for LLM observability an...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.401689,5254bb80-ae8a-489d-a275-055fed5d824b,76555945-7e32-4310-9909-bd39b02eb8f7
3,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,3.203245,5ce69a6a-e69e-4119-83f5-3ea86e5db4a9,7609ba74-8534-4f33-a1db-712e3e3b24a3
4,How do I pass metadata in with @traceable?,You can pass metadata in with `@traceable` by ...,None,You can pass metadata with the @traceable deco...,0,3.475893,61ba4935-6003-4a78-a11a-67439dd8fd54,3af78163-a31e-46b0-8f91-ba2afcb58d13
5,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.173576,8d000be1-4793-4962-894f-772c1809a82a,5bb27000-571e-458a-a9ef-8157225dcf6a
6,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.635373,a868366b-7354-47c8-9f8c-6d46471c0031,3b8617da-47a7-478a-821e-8121846d3b0b
7,Does LangSmith support offline evaluation?,The provided context only mentions online eval...,None,"Yes, LangSmith supports offline evaluation thr...",1,2.045593,a952c4af-ba4b-465f-9f60-e13af2712a6b,6abc8814-4421-46bc-98b0-0eb1a9a78b4f
8,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,2.638593,bf3107ef-3ab7-4672-afc6-e5fac18b6edf,fd33cc3f-074d-4d6b-b2c4-eefdc42e3c14
9,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,1.972232,feca7bf0-9547-470a-b50e-e3797fcafa27,4fed1f32-d6d3-4eb5-a9be-cccc6ddc60fe


In [9]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-370198ca' at:
https://smith.langchain.com/o/0262bc87-4e7f-4047-82e4-1316f23416b0/datasets/5aa34710-6f81-401b-a995-014e4277f4ce/compare?selectedSessions=8ad55b40-2694-4973-86fa-0e1e637a42ea




20it [00:48,  2.45s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.426131,1fe05f92-dff4-4b3d-a0fd-c40cd17a764d,58bb0ada-1ef5-4781-be94-65f3bacd4ffb
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,1.696130,443a5bdb-ddf7-47af-b34f-f8aca324375b,66df2e77-0c52-4a12-9b17-026cc68aebf3
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.626685,5254bb80-ae8a-489d-a275-055fed5d824b,e553a159-cf02-4227-bd5d-46c0c807df0f
3,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Lang...,None,To trace with the @traceable decorator in Pyth...,1,1.573463,5ce69a6a-e69e-4119-83f5-3ea86e5db4a9,c2546c44-3386-4817-8649-8eb5e1b53742
4,How do I pass metadata in with @traceable?,"To pass metadata in with @traceable, you can i...",None,You can pass metadata with the @traceable deco...,1,2.275928,61ba4935-6003-4a78-a11a-67439dd8fd54,1fd93683-2341-4d0f-89d3-8f03f134fd2d
5,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.627802,8d000be1-4793-4962-894f-772c1809a82a,4d785fc9-1661-4f74-9983-5f18336133d5
6,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,2.905950,a868366b-7354-47c8-9f8c-6d46471c0031,c0d925c1-b107-4198-8e81-239260662667
7,Does LangSmith support offline evaluation?,LangSmith documentation does not mention suppo...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.995253,a952c4af-ba4b-465f-9f60-e13af2712a6b,e94217c0-2f1f-4daf-83cd-0c07997ecb08
8,Does LangSmith support online evaluation?,LangSmith supports online evaluations that pro...,None,"Yes, LangSmith supports online evaluation as a...",0,3.984895,bf3107ef-3ab7-4672-afc6-e5fac18b6edf,031520b0-e466-4483-8041-d37f1124dadb
9,What testing capabilities does LangSmith have?,LangSmith in LangChain offers the capability t...,None,LangSmith offers capabilities for creating dat...,1,1.755085,feca7bf0-9547-470a-b50e-e3797fcafa27,4395ebed-f5d1-4949-986a-2c4362889c67


In [11]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name),
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-48048bc8' at:
https://smith.langchain.com/o/0262bc87-4e7f-4047-82e4-1316f23416b0/datasets/5aa34710-6f81-401b-a995-014e4277f4ce/compare?selectedSessions=94e904e2-8ee1-449f-a366-db8bb1564b01




20it [02:55,  8.80s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents,...",None,"Yes, LangSmith can be used to evaluate agents....",0,3.653237,1fe05f92-dff4-4b3d-a0fd-c40cd17a764d,20e8edd1-b717-449e-aa65-6f8b668c7bc5
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,2.095056,443a5bdb-ddf7-47af-b34f-f8aca324375b,7d6540e1-7931-4665-9acb-901f5f80a9fa
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on observabilit...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.578916,5254bb80-ae8a-489d-a275-055fed5d824b,4c308c38-dda6-4bc6-9771-12563a7277a7
3,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, in Pyt...",None,To trace with the @traceable decorator in Pyth...,1,27.668323,5ce69a6a-e69e-4119-83f5-3ea86e5db4a9,bd02087e-7efc-4cd4-a43f-16657a584a04
4,How do I pass metadata in with @traceable?,"To pass metadata in with @traceable, you can u...",None,You can pass metadata with the @traceable deco...,0,2.251672,61ba4935-6003-4a78-a11a-67439dd8fd54,2c6df867-4df6-4d07-a0f4-f2cdaa5207d8
5,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.835769,8d000be1-4793-4962-894f-772c1809a82a,56181ca5-9f65-4712-b21f-14491c01d312
6,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,39.936149,a868366b-7354-47c8-9f8c-6d46471c0031,cedbb843-ef33-4cfe-9ab9-895a127136a9
7,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,3.439268,a952c4af-ba4b-465f-9f60-e13af2712a6b,78a0977a-519f-40c7-bc83-9a9dd3a59087
8,Does LangSmith support online evaluation?,"Yes, LangSmith does support online evaluation ...",None,"Yes, LangSmith supports online evaluation as a...",0,2.873525,bf3107ef-3ab7-4672-afc6-e5fac18b6edf,fbade2a5-5cfd-49a4-b980-ff9e9084a312
9,What testing capabilities does LangSmith have?,LangSmith has testing capabilities that allow ...,None,LangSmith offers capabilities for creating dat...,1,2.509709,feca7bf0-9547-470a-b50e-e3797fcafa27,e3c905f6-4125-4634-8eb8-7c3b5200ba15


In [16]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "572163c6-51cc-48d1-932a-14b0d0fbf189",
            "16038691-db5c-47cb-8918-33e9a6dc30c4"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-b5bd5137' at:
https://smith.langchain.com/o/0262bc87-4e7f-4047-82e4-1316f23416b0/datasets/5aa34710-6f81-401b-a995-014e4277f4ce/compare?selectedSessions=faa439bb-479c-4a47-9cc5-f1fa94a7a8a8




1it [00:11, 11.43s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,3.179763,572163c6-51cc-48d1-932a-14b0d0fbf189,c10f9d83-4544-44ec-a51e-d2c0112e54c6


In [21]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "fe06737f-ae9a-440b-aee1-bfb1441dad11",
            "fc6c0c19-f8e7-4a8f-8e9b-8646bfe92cf4"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-8a6d1e53' at:
https://smith.langchain.com/o/0262bc87-4e7f-4047-82e4-1316f23416b0/datasets/5aa34710-6f81-401b-a995-014e4277f4ce/compare?selectedSessions=b5283414-6ad9-4152-beba-77dc94763f12




1it [00:04,  4.70s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,3.506475,fe06737f-ae9a-440b-aee1-bfb1441dad11,2e12b90e-f58f-4d6b-b562-16acf43bf235


In [20]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "b894201a-78cb-47b1-b84d-010b7a4190d3",
            "b6d78f73-b5dd-4fe7-a254-bc097fdea9e1"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-1ea40ded' at:
https://smith.langchain.com/o/0262bc87-4e7f-4047-82e4-1316f23416b0/datasets/5aa34710-6f81-401b-a995-014e4277f4ce/compare?selectedSessions=e75b757c-248f-4935-a38f-44088d779d75




1it [00:03,  3.89s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,2.460361,b894201a-78cb-47b1-b84d-010b7a4190d3,1acbbc3c-d7bb-48c6-97c3-2f51cc79a034


In [17]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-39ef877c' at:
https://smith.langchain.com/o/0262bc87-4e7f-4047-82e4-1316f23416b0/datasets/5aa34710-6f81-401b-a995-014e4277f4ce/compare?selectedSessions=038443b6-3189-4e08-9dd6-693d9afa2815




40it [01:38,  2.46s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,2.600730,1fe05f92-dff4-4b3d-a0fd-c40cd17a764d,175c715b-7834-4888-a434-860f8dd856d2
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,1,3.015028,443a5bdb-ddf7-47af-b34f-f8aca324375b,f9e37892-3888-404c-b972-2f19df9aef8d
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",0,2.403003,5254bb80-ae8a-489d-a275-055fed5d824b,733c5843-5d1e-4439-9a69-d7f81e032030
3,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.447557,5ce69a6a-e69e-4119-83f5-3ea86e5db4a9,72c81c73-7b93-4a54-bf38-9d1dd1936e9f
4,How do I pass metadata in with @traceable?,You can add metadata to traces with @traceable...,None,You can pass metadata with the @traceable deco...,1,1.892612,61ba4935-6003-4a78-a11a-67439dd8fd54,471c2432-cadf-466c-be69-dbaef6802224
5,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.513645,8d000be1-4793-4962-894f-772c1809a82a,6114b83f-2633-4153-904d-1ddb4253e512
6,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.728219,a868366b-7354-47c8-9f8c-6d46471c0031,d527ba07-8fbe-49a2-8f15-1d74ac0f5dc5
7,Does LangSmith support offline evaluation?,LangSmith does not directly mention support fo...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.736646,a952c4af-ba4b-465f-9f60-e13af2712a6b,1a58bcf0-0643-4f71-8f85-30cf22b62395
8,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,1.615731,bf3107ef-3ab7-4672-afc6-e5fac18b6edf,b6916298-9c4c-45f6-afd1-868ecf77d022
9,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,3.353121,feca7bf0-9547-470a-b50e-e3797fcafa27,77311566-f55b-43ab-8af1-dc19876cecce


In [18]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-5d0a7428' at:
https://smith.langchain.com/o/0262bc87-4e7f-4047-82e4-1316f23416b0/datasets/5aa34710-6f81-401b-a995-014e4277f4ce/compare?selectedSessions=9d19611b-e8fb-433d-8406-2997bcabc66b




20it [00:19,  1.03it/s]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for observabil...,None,"Yes, LangSmith can be used for fine-tuning and...",1,3.086242,5254bb80-ae8a-489d-a275-055fed5d824b,73a63666-7f05-4aa3-ae08-bdfb6eb4b0a5
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,3.183765,443a5bdb-ddf7-47af-b34f-f8aca324375b,81ce2f44-a37d-4bbf-8bcd-e72cac8bd19e
2,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,5.195595,1fe05f92-dff4-4b3d-a0fd-c40cd17a764d,8d3037fe-dcc4-4212-b750-8cbd454c35ae
3,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can in...",None,You can pass metadata with the @traceable deco...,1,3.195854,61ba4935-6003-4a78-a11a-67439dd8fd54,b2b1e2c7-f1ce-41f5-bae4-aaacdf7a29aa
4,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ca...",None,To trace with the @traceable decorator in Pyth...,1,3.444910,5ce69a6a-e69e-4119-83f5-3ea86e5db4a9,c4219acb-9073-40c9-90a1-f6ac29d166ec
5,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.157353,8d000be1-4793-4962-894f-772c1809a82a,39e0b442-5bfd-4987-a833-b06b6c5359c4
6,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,2.070045,a868366b-7354-47c8-9f8c-6d46471c0031,4ef26fc6-b59a-4b10-a150-fe63e86ad283
7,Does LangSmith support offline evaluation?,LangChain's evaluation module in LangSmith pro...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.933473,a952c4af-ba4b-465f-9f60-e13af2712a6b,f414242f-55f6-4ee5-a142-832affff37ef
8,Does LangSmith support online evaluation?,LangSmith supports online evaluation through L...,None,"Yes, LangSmith supports online evaluation as a...",0,2.156530,bf3107ef-3ab7-4672-afc6-e5fac18b6edf,a67dc926-ca68-4d6a-b0da-041dc467d6e9
9,What testing capabilities does LangSmith have?,LangSmith allows running multiple experiments ...,None,LangSmith offers capabilities for creating dat...,1,1.509086,feca7bf0-9547-470a-b50e-e3797fcafa27,f8b86548-efd5-42ed-b82d-d7eee9727790


In [19]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME 
    }
)

View the evaluation results for experiment: 'metadata added-2eaa10bc' at:
https://smith.langchain.com/o/0262bc87-4e7f-4047-82e4-1316f23416b0/datasets/5aa34710-6f81-401b-a995-014e4277f4ce/compare?selectedSessions=dc49f906-0169-4b02-86c4-2c6af13b8bf0




20it [00:41,  2.06s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.738728,1fe05f92-dff4-4b3d-a0fd-c40cd17a764d,46d73d26-be15-443d-a0c9-7786ca1bff25
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,3.759729,443a5bdb-ddf7-47af-b34f-f8aca324375b,cdaf3858-4990-46b5-97bc-3b53214e9260
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.852715,5254bb80-ae8a-489d-a275-055fed5d824b,ba79457d-9af1-4d76-ab3c-d9f769bc8770
3,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.765790,5ce69a6a-e69e-4119-83f5-3ea86e5db4a9,a3f8ef8e-4cc5-4cea-b24c-3cc553d3538b
4,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can in...",None,You can pass metadata with the @traceable deco...,1,1.749396,61ba4935-6003-4a78-a11a-67439dd8fd54,61473c2b-8554-4ad4-95ce-7244c76ab85e
5,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.686937,8d000be1-4793-4962-894f-772c1809a82a,90c7808b-aad6-4eaa-9a3f-98c69d79a67b
6,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.721256,a868366b-7354-47c8-9f8c-6d46471c0031,6338181f-fbe1-4654-9aac-bad49f445ae3
7,Does LangSmith support offline evaluation?,LangSmith mainly supports online evaluation me...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.740224,a952c4af-ba4b-465f-9f60-e13af2712a6b,4e8a0d27-c37e-4a22-b224-ec44db0a7f76
8,Does LangSmith support online evaluation?,"Yes, LangSmith does support online evaluation ...",None,"Yes, LangSmith supports online evaluation as a...",0,1.418311,bf3107ef-3ab7-4672-afc6-e5fac18b6edf,96b9c1aa-ec46-4ea5-97b4-929b7d56d2b8
9,What testing capabilities does LangSmith have?,LangSmith offers the capability to run multipl...,None,LangSmith offers capabilities for creating dat...,1,1.324413,feca7bf0-9547-470a-b50e-e3797fcafa27,f383f6e2-7d24-4f31-a6d9-b4d4e8fbde60
